In [1]:
import mytools
import pandas as pd
import geopandas as gpd
import simplekml

In [6]:
data =pd.ExcelFile('C:/Users/Administrator/Desktop/2020任务清单.xlsx')

In [7]:
data.sheet_names

['新', '商俊']

In [8]:
data_2020 = data.parse(sheet_name='新')
data_shangjun = data.parse(sheet_name='商俊')

In [14]:
data_2020_sue = data_2020[['唯一编号','lon', 'lat']]

In [15]:
data_shangjun_use = data_shangjun[['唯一编号', '问题点中心经度', '问题点中心纬度']]

In [21]:
data_shangjun_use['制式']='FDD900'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
res = mytools.gisn.points_coverage_merge(data_shangjun_use ,'问题点中心经度', '问题点中心纬度')

In [23]:
data_2020_p = mytools.gisn.add_points(data_2020_sue)

In [24]:
res.head(1)

,唯一编号,问题点中心经度,问题点中心纬度,制式,区域类型
0,69265,113.162358,30.652183,FDD900,主城区


In [25]:
res_distance = mytools.gisn.changjing_distance(res,'区域类型','制式')

In [26]:
res_distance.head(1)

,唯一编号,问题点中心经度,问题点中心纬度,制式,区域类型,distance
0,69265,113.162358,30.652183,FDD900,主城区,300.0


In [27]:
res_buff = mytools.gisn.add_buffer_df(res_distance,'问题点中心经度', '问题点中心纬度','distance')

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [28]:
guanlian = gpd.sjoin(res_buff,data_2020_p)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [29]:
guanlian.drop(columns='geometry').to_csv('C:/Users/Administrator/Desktop/商俊任务清单关联2020.csv',encoding='gbk',index = False)

In [52]:
kml = simplekml.Kml()
style = simplekml.Style()
style.polystyle.color = simplekml.Color.changealphaint(125, 'ff00a5ff' )
for name,geo in zip(res_buff['唯一编号'],res_buff['geometry']):
    pol = kml.newpolygon(name=str(name))
    pol.description = str(name)
    pol.outerboundaryis=list(geo.exterior.coords)
    pol.style = style

In [53]:
kml.save('d:/a_buff.kml')
print('ok')

ok


In [41]:
data_2020_p.head(1)

,唯一编号,lon,lat,geometry
0,100994,108.40894,30.380615,POINT (108.40894 30.38062)


In [48]:
kml = simplekml.Kml()
style = simplekml.Style()
for name,geo in zip(data_2020_p['唯一编号'],data_2020_p['geometry']):
    pol = kml.newpoint(name=str(name))
    pol.coords=list(geo.coords)
    pol.style = style

In [49]:
kml.save('d:/a_dian.kml')
print('ok')

ok
